In [1]:
!pip install image_classifiers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential, Input
from keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from keras.applications.resnet_v2 import preprocess_input
from keras.models import Model
from classification_models.keras import Classifiers
from keras.optimizers import Adam
from keras.activations import linear
from tqdm.notebook import tqdm
from keras.utils import Progbar
from keras.backend import clear_session
from tensorflow.nn import softmax_cross_entropy_with_logits
import os

2024-04-17 11:13:36.930323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 11:13:38.235404: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 40
batch_size = 64
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant =result Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.0 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 111.90 MiB/s 1s

2024-04-15 14:33:13.034406: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


In [5]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def train_val_split(train_dataset, validation_size):
    X_train, y_train = train_dataset
    indices = np.random.permutation(X_train.shape[0])
    train_idx, val_idx = indices[:train_size], indices[train_size:]
    return (X_train[train_idx,...], y_train[train_idx,...]), (X_train[val_idx,...], y_train[val_idx,...])

def load_cifar_train_val():
    train_dataset, _ = cifar100.load_data()
    
    train_dataset, validation_dataset = train_val_split(train_dataset, validation_size)
    
    validation_dataset = tf.data.Dataset.from_tensor_slices(validation_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet).cache().prefetch(tf.data.AUTOTUNE)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(train_size).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, validation_dataset

In [11]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

@tf.function
def ce(x, y_logits, temp):
    return softmax_cross_entropy_with_logits(x, y_logits / temp) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy,val_accuracy\n")
def append_csv_log(fichier, epoch, accuracy,val_accuracy):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy:.2f},{val_accuracy:.2f}\n")

def distillateur_kl(etudiant, enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha):
    @tf.function
    def forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, adam):
        X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
        with tf.GradientTape() as tape:
            etudiant_estim_logit = etudiant_logit_model(X_batch, training = True)
            perte = alpha * softmax_cross_entropy_with_logits(y_batch,etudiant_estim_logit) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp)
        grads = tape.gradient(perte, etudiant_logit_model.trainable_variables)
        adam.apply_gradients(zip(grads, etudiant_logit_model.trainable_variables))
        return compte_bons(etudiant_estim_logit,y_batch)
        # La fonction ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    etudiant_logit_model = get_modele_logits(etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    adam = Adam(learning_rate=0.001)
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    val_accuracy_max = 0
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        n_batch = train_size//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc"])
        bons_epoque = 0
        for i in range(n_batch):
            bons_epoque += forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, adam).numpy()
            accuracy = bons_epoque / ((i+1) * batch_size)
            barre_progression.update(i + 1, values = [("acc", accuracy)])
        _, val_accuracy = etudiant.evaluate(validation_dataset, verbose = 0)
        if val_accuracy > val_accuracy_max:
            val_accuracy_max = val_accuracy
            etudiant.save(f"sauvegardes/{nom_modele}_checkpoint.keras")
        else:
            adam.learning_rate.assign(adam.learning_rate.numpy() * 0.8)
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy, val_accuracy)
        print(f"Accuracy (train) : {accuracy:.4f} | Accuracy (val) : {val_accuracy:.4f}")

In [7]:
def distillation_resnet18(temp, alpha):
    clear_session()
    train_dataset, validation_dataset = load_cifar_train_val()
    modele = new_modele_resnet()
    nom_modele =  f"model_etudiant_t{temp:d}_a{int(alpha*100):d}"
    distillateur_kl(modele, model_enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch,0.25)
    wd = os.getcwd()
    os.system(f"cp {wd}/sauvegardes/{nom_modele}_checkpoint.keras {wd}/sauvegardes/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

In [8]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [8]:
distillation_resnet18(1,1) # témoin

44920640/44920640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
C'est parti pour la distillation !

Époque 1 / 40


2024-04-14 07:20:49.573939: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 440s 529ms/step - acc: 0.1274


I0000 00:00:1713079709.476407     499 service.cc:145] XLA service 0x7f8f8801ce70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713079709.477075     499 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-14 07:28:29.590843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713079715.984703     499 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1274 | Accuracy (val) : 0.0906
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 529ms/step - acc: 0.2682
Accuracy (train) : 0.2682 | Accuracy (val) : 0.2848
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 403s 574ms/step - acc: 0.3493
Accuracy (train) : 0.3493 | Accuracy (val) : 0.3452
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 610ms/step - acc: 0.4071
Accuracy (train) : 0.4071 | Accuracy (val) : 0.3912
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 400s 570ms/step - acc: 0.4442
Accuracy (train) : 0.4442 | Accuracy (val) : 0.4294
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 603ms/step - acc: 0.4709
Accuracy (train) : 0.4709 | Accuracy (val) : 0.4556
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 416s 593ms/step - acc: 0.4921
Accuracy (train) : 0.4921 | Accuracy (val) : 0.4596
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.5160
Accuracy (train) : 0.5160 | Accuracy (val) : 0.4298
Époque 9 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.5510


2024-04-14 12:06:43.900373: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-14 12:06:43.900556: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[sequential_1_1/random_translation_1/add_1/_42]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-14 12:06:43.900630: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7564607746336802277
2024-04-14 12:06:43.900694: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_rotation_1/add/_32}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-1

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a100.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a100.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 106.25 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a100_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a100_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.00 KiB/s


In [8]:
distillation_resnet18(1,0.25)

C'est parti pour la distillation !

Époque 1 / 40


2024-04-14 12:23:51.661655: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 456s 599ms/step - acc: 0.1414


I0000 00:00:1713097902.336686    5895 service.cc:145] XLA service 0x7f2da0e4f130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713097902.336907    5895 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-14 12:31:42.593707: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713097908.909483    5895 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1414 | Accuracy (val) : 0.1056
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 379s 539ms/step - acc: 0.2827
Accuracy (train) : 0.2827 | Accuracy (val) : 0.2822
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 608ms/step - acc: 0.3569
Accuracy (train) : 0.3569 | Accuracy (val) : 0.3660
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 597ms/step - acc: 0.4121
Accuracy (train) : 0.4121 | Accuracy (val) : 0.3900
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 608ms/step - acc: 0.4448
Accuracy (train) : 0.4448 | Accuracy (val) : 0.4198
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 600ms/step - acc: 0.4772
Accuracy (train) : 0.4772 | Accuracy (val) : 0.4476
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 416s 592ms/step - acc: 0.5010
Accuracy (train) : 0.5010 | Accuracy (val) : 0.4516
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 435s 618ms/step - acc: 0.5221
Accuracy (train) : 0.5221 | Accuracy (val) : 0.4648
Époque 9 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 587ms/step - acc: 0.5376


2024-04-14 17:12:29.723072: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-14 17:12:29.723268: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_rotation_1/add/_32}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-14 17:12:29.723295: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_rotation_1/add/_32}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_45]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-14 17:12:29.723324: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10989029791097413689
2024-04-14 17

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a25.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a25.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 124.40 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a25_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a25_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 5.21 KiB/s


In [8]:
distillation_resnet18(3,0.25)

C'est parti pour la distillation !

Époque 1 / 40


2024-04-14 22:35:25.399786: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 451s 547ms/step - acc: 0.1601


I0000 00:00:1713134594.625525   11999 service.cc:145] XLA service 0x7f1828008920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713134594.626286   11999 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-14 22:43:14.738258: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713134601.163121   11999 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1601 | Accuracy (val) : 0.1028
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 381s 542ms/step - acc: 0.3106
Accuracy (train) : 0.3106 | Accuracy (val) : 0.3100
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 604ms/step - acc: 0.3890
Accuracy (train) : 0.3890 | Accuracy (val) : 0.4018
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 613ms/step - acc: 0.4369
Accuracy (train) : 0.4369 | Accuracy (val) : 0.4354
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.4700
Accuracy (train) : 0.4700 | Accuracy (val) : 0.4630
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 612ms/step - acc: 0.4995
Accuracy (train) : 0.4995 | Accuracy (val) : 0.4986
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 611ms/step - acc: 0.5203
Accuracy (train) : 0.5203 | Accuracy (val) : 0.4916
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 441s 628ms/step - acc: 0.5599
Accuracy (train) : 0.5599 | Accuracy (val) : 0.5248
Époque 9 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.5743


2024-04-15 03:27:02.176839: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-15 03:27:02.176973: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[sequential_1_1/random_zoom_1/add/_36]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-15 03:27:02.177054: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13255652756699139431
2024-04-15 03:27:02.177180: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_contrast_1/add/_28}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-15 03:27:

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a25.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a25.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 58.82 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a25_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a25_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 2.14 KiB/s


In [9]:
distillation_resnet18(8,0.25)

C'est parti pour la distillation !

Époque 1 / 40


2024-04-15 06:17:52.417181: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 466s 613ms/step - acc: 0.1558


I0000 00:00:1713162353.081146   18572 service.cc:145] XLA service 0x7fd588021560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713162353.081313   18572 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-15 06:25:53.267034: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713162359.468537   18572 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1558 | Accuracy (val) : 0.1616
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 381s 543ms/step - acc: 0.3043
Accuracy (train) : 0.3043 | Accuracy (val) : 0.3044
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 432s 615ms/step - acc: 0.3763
Accuracy (train) : 0.3763 | Accuracy (val) : 0.3824
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 418s 595ms/step - acc: 0.4290
Accuracy (train) : 0.4290 | Accuracy (val) : 0.4136
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 611ms/step - acc: 0.4641
Accuracy (train) : 0.4641 | Accuracy (val) : 0.4388
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 418s 594ms/step - acc: 0.4931
Accuracy (train) : 0.4931 | Accuracy (val) : 0.4526
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.5154
Accuracy (train) : 0.5154 | Accuracy (val) : 0.4732
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 617ms/step - acc: 0.5358
Accuracy (train) : 0.5358 | Accuracy (val) : 0.4766
Époque 9 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 426s 607ms/step - acc: 0.5577


2024-04-15 11:09:43.058042: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-15 11:09:44.391824: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_44]] [type.googleapis.com/tensorflow.DerivedStatus='']


`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a25.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a25.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 109.44 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a25_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a25_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.57 KiB/s


In [9]:
distillation_resnet18(1,0.5)

C'est parti pour la distillation !

Époque 1 / 40


2024-04-15 14:33:55.529481: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 456s 597ms/step - acc: 0.1438


I0000 00:00:1713192107.609898   24022 service.cc:145] XLA service 0x7f7b7c003960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713192107.610089   24022 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-15 14:41:47.750473: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713192115.907238   24022 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1438 | Accuracy (val) : 0.1078
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 532ms/step - acc: 0.2846
Accuracy (train) : 0.2846 | Accuracy (val) : 0.2800
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 416s 592ms/step - acc: 0.3668
Accuracy (train) : 0.3668 | Accuracy (val) : 0.3954
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 409s 582ms/step - acc: 0.4132
Accuracy (train) : 0.4132 | Accuracy (val) : 0.4126
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 600ms/step - acc: 0.4484
Accuracy (train) : 0.4484 | Accuracy (val) : 0.4322
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 413s 588ms/step - acc: 0.4764
Accuracy (train) : 0.4764 | Accuracy (val) : 0.4614
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 425s 605ms/step - acc: 0.5034
Accuracy (train) : 0.5034 | Accuracy (val) : 0.4758
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 425s 605ms/step - acc: 0.5236
Accuracy (train) : 0.5236 | Accuracy (val) : 0.4802
Époque 9 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.5421


In [12]:
distillation_resnet18(3,0.5)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 461s 614ms/step - acc: 0.1544
Accuracy (train) : 0.1544 | Accuracy (val) : 0.0870
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 544ms/step - acc: 0.2986
Accuracy (train) : 0.2986 | Accuracy (val) : 0.2714
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 611ms/step - acc: 0.3770
Accuracy (train) : 0.3770 | Accuracy (val) : 0.3814
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 604ms/step - acc: 0.4293
Accuracy (train) : 0.4293 | Accuracy (val) : 0.3968
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 608ms/step - acc: 0.4684
Accuracy (train) : 0.4684 | Accuracy (val) : 0.4566
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 609ms/step - acc: 0.4999
Accuracy (train) : 0.4999 | Accuracy (val) : 0.4684
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.5172
Accuracy (train) : 0.5172 | Accuracy (val) : 0.4966
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 426s 607ms/step - acc: 0.5407
Accuracy (train)

2024-04-16 00:33:50.682953: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add/_34}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-16 00:33:50.683195: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 2268825309413901516
2024-04-16 00:33:50.683220: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 10674578115712132140
2024-04-16 00:33:50.683252: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6430510491586721336
2024-04-16 00:33:50.683263: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 5457321438645917642
2024-04-16 00:33:50.683278: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 1448897

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a50.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a50.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 83.49 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a50_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a50_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.65 KiB/s


In [13]:
distillation_resnet18(8,0.5)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 459s 610ms/step - acc: 0.1466
Accuracy (train) : 0.1466 | Accuracy (val) : 0.1244
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 370s 527ms/step - acc: 0.2937
Accuracy (train) : 0.2937 | Accuracy (val) : 0.3092
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.3710
Accuracy (train) : 0.3710 | Accuracy (val) : 0.3298
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 590ms/step - acc: 0.4177
Accuracy (train) : 0.4177 | Accuracy (val) : 0.4036
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 413s 587ms/step - acc: 0.4578
Accuracy (train) : 0.4578 | Accuracy (val) : 0.4412
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 416s 592ms/step - acc: 0.4899
Accuracy (train) : 0.4899 | Accuracy (val) : 0.4414
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 417s 594ms/step - acc: 0.5148
Accuracy (train) : 0.5148 | Accuracy (val) : 0.4440
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.5320
Accuracy (train)

In [14]:
distillation_resnet18(1,0)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 469s 626ms/step - acc: 0.1338
Accuracy (train) : 0.1338 | Accuracy (val) : 0.1440
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 363s 516ms/step - acc: 0.2828
Accuracy (train) : 0.2828 | Accuracy (val) : 0.3186
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 598ms/step - acc: 0.3604
Accuracy (train) : 0.3604 | Accuracy (val) : 0.3518
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 438s 623ms/step - acc: 0.4035
Accuracy (train) : 0.4035 | Accuracy (val) : 0.4116
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 598ms/step - acc: 0.4416
Accuracy (train) : 0.4416 | Accuracy (val) : 0.4210
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.4748
Accuracy (train) : 0.4748 | Accuracy (val) : 0.4410
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 607ms/step - acc: 0.5036
Accuracy (train) : 0.5036 | Accuracy (val) : 0.4802
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 609ms/step - acc: 0.5163
Accuracy (train)

2024-04-16 10:08:14.821709: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add_1/_42}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-16 10:08:14.821825: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 10674578115712132140
2024-04-16 10:08:14.821844: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 2268825309413901516
2024-04-16 10:08:14.821862: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 14488976433331128082
2024-04-16 10:08:14.821886: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 8876865378681747252
2024-04-16 10:08:14.821901: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 5457

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a0.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a0.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 113.10 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a0_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a0_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.11 KiB/s


In [15]:
distillation_resnet18(3,0)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 465s 600ms/step - acc: 0.1611
Accuracy (train) : 0.1611 | Accuracy (val) : 0.1380
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 371s 528ms/step - acc: 0.3120
Accuracy (train) : 0.3120 | Accuracy (val) : 0.3304
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.3906
Accuracy (train) : 0.3906 | Accuracy (val) : 0.3718
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 416s 592ms/step - acc: 0.4375
Accuracy (train) : 0.4375 | Accuracy (val) : 0.3998
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 600ms/step - acc: 0.4766
Accuracy (train) : 0.4766 | Accuracy (val) : 0.4588
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 617ms/step - acc: 0.5037
Accuracy (train) : 0.5037 | Accuracy (val) : 0.4670
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.5249
Accuracy (train) : 0.5249 | Accuracy (val) : 0.5198
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 388s 553ms/step - acc: 0.5423
Accuracy (train)

In [16]:
distillation_resnet18(8,0)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 490s 630ms/step - acc: 0.1497
Accuracy (train) : 0.1497 | Accuracy (val) : 0.1186
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 384s 546ms/step - acc: 0.2932
Accuracy (train) : 0.2932 | Accuracy (val) : 0.3334
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 613ms/step - acc: 0.3713
Accuracy (train) : 0.3713 | Accuracy (val) : 0.3772
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 601ms/step - acc: 0.4180
Accuracy (train) : 0.4180 | Accuracy (val) : 0.3990
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 617ms/step - acc: 0.4574
Accuracy (train) : 0.4574 | Accuracy (val) : 0.4052
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 590ms/step - acc: 0.4895
Accuracy (train) : 0.4895 | Accuracy (val) : 0.4258
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 607ms/step - acc: 0.5108
Accuracy (train) : 0.5108 | Accuracy (val) : 0.4356
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 617ms/step - acc: 0.5285
Accuracy (train)

2024-04-16 19:59:36.181010: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add/_34}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-16 19:59:36.181667: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add/_34}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_45]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-16 19:59:36.181738: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2179606093766412807
2024-04-16 19:59:36.181770: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17239033057816435435
2024-04-16 19:59:36.181815: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Ke

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a0.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a0.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 137.09 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a0_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a0_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.64 KiB/s


In [17]:
distillation_resnet18(1,0.1)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 495s 639ms/step - acc: 0.1366
Accuracy (train) : 0.1366 | Accuracy (val) : 0.0622
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 385s 548ms/step - acc: 0.2858
Accuracy (train) : 0.2858 | Accuracy (val) : 0.2912
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 601ms/step - acc: 0.3593
Accuracy (train) : 0.3593 | Accuracy (val) : 0.3588
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 611ms/step - acc: 0.4121
Accuracy (train) : 0.4121 | Accuracy (val) : 0.3892
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 432s 614ms/step - acc: 0.4471
Accuracy (train) : 0.4471 | Accuracy (val) : 0.4130
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 600ms/step - acc: 0.4789
Accuracy (train) : 0.4789 | Accuracy (val) : 0.4342
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 418s 595ms/step - acc: 0.4980
Accuracy (train) : 0.4980 | Accuracy (val) : 0.4548
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 600ms/step - acc: 0.5219
Accuracy (train)

2024-04-17 00:53:59.855531: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add_1/_42}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 00:53:59.855770: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/add_1/_42}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_45]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 00:53:59.855814: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 4898638260826039205
2024-04-17 00:53:59.855827: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 3320248784942991097
2024-04-17 00:53:59.855890: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled.

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 142.15 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a10_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 3.71 KiB/s


In [18]:
distillation_resnet18(3,0.1)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 496s 635ms/step - acc: 0.1432
Accuracy (train) : 0.1432 | Accuracy (val) : 0.1008
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 394s 560ms/step - acc: 0.2901
Accuracy (train) : 0.2901 | Accuracy (val) : 0.3038
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 438s 623ms/step - acc: 0.3736
Accuracy (train) : 0.3736 | Accuracy (val) : 0.3682
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 433s 616ms/step - acc: 0.4196
Accuracy (train) : 0.4196 | Accuracy (val) : 0.3884
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 614ms/step - acc: 0.4624
Accuracy (train) : 0.4624 | Accuracy (val) : 0.4532
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 437s 622ms/step - acc: 0.4949
Accuracy (train) : 0.4949 | Accuracy (val) : 0.4862
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 612ms/step - acc: 0.5148
Accuracy (train) : 0.5148 | Accuracy (val) : 0.5010
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 435s 619ms/step - acc: 0.5318
Accuracy (train)

2024-04-17 05:49:42.914426: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_contrast_1/add/_28}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 05:49:42.922218: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_contrast_1/add/_28}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_45]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 05:49:42.957713: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6439522439796177520
2024-04-17 05:49:42.957821: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6430510491586721336
2024-04-17 05:49:42.957937: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCE

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 105.21 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a10_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 3.75 KiB/s


In [19]:
distillation_resnet18(8,0.1)

C'est parti pour la distillation !

Époque 1 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 485s 632ms/step - acc: 0.1534
Accuracy (train) : 0.1534 | Accuracy (val) : 0.1096
Époque 2 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 387s 551ms/step - acc: 0.2938
Accuracy (train) : 0.2938 | Accuracy (val) : 0.2292
Époque 3 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 618ms/step - acc: 0.3742
Accuracy (train) : 0.3742 | Accuracy (val) : 0.3446
Époque 4 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 601ms/step - acc: 0.4229
Accuracy (train) : 0.4229 | Accuracy (val) : 0.3882
Époque 5 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 601ms/step - acc: 0.4581
Accuracy (train) : 0.4581 | Accuracy (val) : 0.3878
Époque 6 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 613ms/step - acc: 0.5017
Accuracy (train) : 0.5017 | Accuracy (val) : 0.4564
Époque 7 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 612ms/step - acc: 0.5278
Accuracy (train) : 0.5278 | Accuracy (val) : 0.4760
Époque 8 / 40
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 601ms/step - acc: 0.5514
Accuracy (train)

2024-04-17 10:42:12.981877: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_zoom_1/add/_36}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 10:42:12.982230: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 2268825309413901516
2024-04-17 10:42:12.982258: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 10674578115712132140
2024-04-17 10:42:12.982306: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6430510491586721336
2024-04-17 10:42:12.982319: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 9641515304282884847
2024-04-17 10:42:12.982331: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 48986382608260

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 69.39 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a10_logs.csv`
Total: 539 B, Transferred: 539 B, Speed: 4.25 KiB/s
